# Analyzing data

In [ ]:
from dbpal.tables import tbls
from siuba import *

## Get information about a table

Print out a table to get information about it's column types, and descriptions (if available).

In [ ]:
tbls.stg_issues

## Querying a table

Call the table to get back a [siuba LazyTbl](https://siuba.org/guide/basics-sql.html), which can be wrangled dbplyr style:

### Example: simple issue count

In [ ]:
tbls.stg_issues() >> count()

### Example: counting issues per repo

We could count the number of issues per repo with two steps:

* Joining `stg_issues.repository_id` with `stg_repository.id`, in order to get each repositories name.
* Counting repository name.

This is shown in the code below.

In [ ]:
(
    tbls.stg_issues()
    >> inner_join(
        _,
        tbls.stg_repository() >> select(_.repository_id == _.id, _.full_name),
        "repository_id"
    )
    >> count(_.full_name)
)

## Query using raw sql

Use `tbls.query()` to query using raw sql, and return the results as a siuba `LazyTbl`.

For example, the query below counts daily siuba downloads from pypi, using the public bigquery dataset `bigquery-public-data.pypi.file_downloads`:

In [ ]:
tbls.query("""
SELECT
  CAST(file_downloads.timestamp  AS DATE) AS file_downloads_timestamp_date,
       file_downloads.file.project  AS file_downloads_file__project,
       COUNT(*) AS file_downloads_count
FROM `bigquery-public-data.pypi.file_downloads`
  AS file_downloads
WHERE 
    -- filter siuba downloads in the last 30 days
    (file_downloads.file.project = 'siuba')
    AND (file_downloads.timestamp >= timestamp_add(current_timestamp(), INTERVAL -30 DAY)) 
GROUP BY 1,2
ORDER BY 1,2,3
""")